In [17]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import (
    mean_absolute_error,
    mean_squared_error,
    mean_absolute_percentage_error,
)

import tensorflow as tf
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import Input, LSTM, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Quality Variables

- Gesamtes abgelaufenes Volumen (Für jeden Batch die kumulative Summe von v(t))
- Durchschnittlicher Füllstand von Tank 2 (jeden Batch den Mittelwert der Spalte x2(t))

In [2]:
res = pd.read_csv("3Tank_Experiment_Batch.csv")

In [3]:
res

,t,Sequenz,u1(t),u2(t),x1(t),x2(t),x3(t),v(t)
0,0,1,0.000000,0.000000,4.842331,4.010093,4.561483,9.364711
1,1,1,5.361920,3.086492,4.651301,4.667351,4.650984,9.151476
2,2,1,9.621660,5.754046,4.503403,4.278377,4.703018,9.224275
3,3,1,12.934532,8.040007,4.714889,4.527859,4.541644,9.345350
4,4,1,15.444613,9.980081,4.884475,4.885002,4.768943,9.949724
...,...,...,...,...,...,...,...,...
27995,135,200,25.000000,6.088392,7.664921,6.772946,0.878545,4.099904
27996,136,200,25.000000,5.493404,7.846334,7.237114,1.043541,4.142338
27997,137,200,25.000000,4.855503,7.927747,7.172953,0.864453,3.404311
27998,138,200,25.000000,4.172510,8.215693,7.258894,0.856796,3.257938


In [13]:
px.line(res, x="t", y="v(t)", color="Sequenz", title="Füllstand v(t) über die Zeit")

In [4]:
# QV-ENGINEERING

res["V_total_out"] = res.groupby("Sequenz")["v(t)"].transform("sum")

In [5]:
# DATEN-PIPELINE

features = ["u1(t)", "u2(t)", "v(t)", "x1(t)", "x2(t)"]
target = "V_total_out"

print(f"Features: {features}")
print(f"Target: {target}")


# Daten in Sequenzen umwandeln
unique_sequences = res["Sequenz"].unique()
n_sequences = len(unique_sequences)
n_timesteps = res.groupby("Sequenz").size().iloc[0]
n_features = len(features)

# Initialisiere die leeren NumPy-Arrays
X = np.zeros((n_sequences, n_timesteps, n_features))
y = np.zeros((n_sequences, 1))

print(f"\nErstelle 3D-Tensoren mit der Form (Batches, Zeitschritte, Features)...")
for i, seq_id in enumerate(unique_sequences):
    # Extrahiere die Features für die aktuelle Sequenz
    sequence_data = res[res["Sequenz"] == seq_id][features].values
    X[i] = sequence_data

    # Extrahiere den Zielwert (ist für die ganze Sequenz gleich, also nehmen wir den ersten)
    target_value = res[res["Sequenz"] == seq_id][target].iloc[0]
    y[i] = target_value

print(f"Form des Feature-Tensors X: {X.shape}")
print(f"Form des Target-Vektors y: {y.shape}")


# 3. Aufteilen in Trainings-, Validierungs- und Testdaten (Batch-weise)
# Wir teilen die Indizes der Batches auf, nicht die einzelnen Zeilen!
indices = np.arange(n_sequences)

# Zuerst 80% Training und 20% für (Validierung + Test)
X_train, X_temp, y_train, y_temp, indices_train, indices_temp = train_test_split(
    X, y, indices, test_size=0.2, random_state=42
)

# Dann die 20% in 10% Validierung und 10% Test aufteilen (50/50 split von temp)
X_val, X_test, y_val, y_test, indices_val, indices_test = train_test_split(
    X_temp, y_temp, indices_temp, test_size=0.5, random_state=42
)

print("\nDaten aufgeteilt:")
print(f"Trainingsdaten: {X_train.shape[0]} Batches")
print(f"Validierungsdaten: {X_val.shape[0]} Batches")
print(f"Testdaten: {X_test.shape[0]} Batches")


# 4. Skalierung der Daten

# Skalierung für die Features (X)
scaler_X = StandardScaler()
X_train_reshaped = X_train.reshape(-1, n_features)
scaler_X.fit(X_train_reshaped)

# Transformation anwenden und zurück in 3D-Form bringen
X_train_scaled = scaler_X.transform(X_train_reshaped).reshape(X_train.shape)
X_val_scaled = scaler_X.transform(X_val.reshape(-1, n_features)).reshape(X_val.shape)
X_test_scaled = scaler_X.transform(X_test.reshape(-1, n_features)).reshape(X_test.shape)

# Skalierung für den Zielwert (y)
scaler_y = StandardScaler()
y_train_scaled = scaler_y.fit_transform(y_train)
y_val_scaled = scaler_y.transform(y_val)
y_test_scaled = scaler_y.transform(y_test)


print(f"X_train_scaled: {X_train_scaled.shape}, y_train_scaled: {y_train_scaled.shape}")
print(f"X_val_scaled: {X_val_scaled.shape}, y_val_scaled: {y_val_scaled.shape}")
print(f"X_test_scaled: {X_test_scaled.shape}, y_test_scaled: {y_test_scaled.shape}")

Features: ['u1(t)', 'u2(t)', 'v(t)', 'x1(t)', 'x2(t)']
Target: V_total_out

Erstelle 3D-Tensoren mit der Form (Batches, Zeitschritte, Features)...
Form des Feature-Tensors X: (200, 140, 5)
Form des Target-Vektors y: (200, 1)

Daten aufgeteilt:
Trainingsdaten: 160 Batches
Validierungsdaten: 20 Batches
Testdaten: 20 Batches
X_train_scaled: (160, 140, 5), y_train_scaled: (160, 1)
X_val_scaled: (20, 140, 5), y_val_scaled: (20, 1)
X_test_scaled: (20, 140, 5), y_test_scaled: (20, 1)


In [ ]:
# MODELL-PROTOTYPING (LSTM)

# Funktion zum Erstellen des Modells
def build_lstm_model(input_shape):
    """Baut und kompiliert ein LSTM-Modell für die Sequenz-zu-Wert-Regression."""

    # Input-Layer, der die Form unserer Sequenzen erwartet
    inputs = Input(shape=input_shape)

    # LSTM-Layer
    lstm_out = LSTM(units=64, return_sequences=False)(inputs)

    # Output-Layer
    outputs = Dense(units=1)(lstm_out)

    # Erstelle das Modell
    model = Model(inputs=inputs, outputs=outputs)

    # Kompiliere das Modell
    model.compile(
        optimizer=Adam(learning_rate=0.001),
        loss="mean_squared_error",
        metrics=["mean_absolute_error"],
    )

    return model


# Definiere die Input-Form basierend auf unseren aufbereiteten Daten
input_shape = (X_train_scaled.shape[1], X_train_scaled.shape[2])  # (140, 5)

# Baue das Modell und zeige die Architektur an
model = build_lstm_model(input_shape)
print("Modellarchitektur:")
model.summary()


# Callbacks für ein besseres Training
# 1. EarlyStopping: Beendet das Training, wenn die Validierungs-Loss sich nicht mehr verbessert.
early_stopping = EarlyStopping(
    monitor="val_loss",  # Überwache die Loss auf den Validierungsdaten
    patience=10,  # Anzahl der Epochen ohne Verbesserung, bevor gestoppt wird
    verbose=1,
    restore_best_weights=True,
)  # Stellt die besten Gewichte am Ende wieder her

# 2. ModelCheckpoint: Speichert das beste Modell während des Trainings.
model_checkpoint = ModelCheckpoint(
    "best_model.keras",  # Dateipfad
    monitor="val_loss",
    save_best_only=True,  # Speichere nur, wenn 'val_loss' sich verbessert
    verbose=1,
)

# Trainiere das Modell
print("\nStarte das Modelltraining...")
history = model.fit(
    X_train_scaled,
    y_train_scaled,
    epochs=50,
    batch_size=32,
    validation_data=(X_val_scaled, y_val_scaled),
    callbacks=[early_stopping, model_checkpoint],
    verbose=1,
)
print("Modelltraining abgeschlossen.")

Modellarchitektur:


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 140, 5)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ (None, 64)             │        17,920 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 17,985 (70.25 KB)

 Trainable params: 17,985 (70.25 KB)

 Non-trainable params: 0 (0.00 B)


Starte das Modelltraining...
Epoch 1/50
1/5 ━━━━━━━━━━━━━━━━━━━━ 1s 372ms/step - loss: 1.0383 - mean_absolute_error: 0.8563
Epoch 1: val_loss improved from inf to 0.78037, saving model to best_model.keras
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 40ms/step - loss: 0.9061 - mean_absolute_error: 0.7837 - val_loss: 0.7804 - val_mean_absolute_error: 0.7091
Epoch 2/50
1/5 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - loss: 0.9677 - mean_absolute_error: 0.8076
Epoch 2: val_loss improved from 0.78037 to 0.69361, saving model to best_model.keras
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step - loss: 0.7384 - mean_absolute_error: 0.6959 - val_loss: 0.6936 - val_mean_absolute_error: 0.6959
Epoch 3/50
1/5 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - loss: 0.3818 - mean_absolute_error: 0.4849
Epoch 3: val_loss improved from 0.69361 to 0.51369, saving model to best_model.keras
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step - loss: 0.5307 - mean_absolute_error: 0.5873 - val_loss: 0.5137 - val_mean_absolute_error: 0.5833
Epoch 4/50
1/5 ━━━━━━━━━━━━━

In [8]:
# VISUALISIERUNG DES TRAININGSVERLAUFS

# Erstelle einen DataFrame aus der Trainingshistorie
history_df = pd.DataFrame(history.history)
history_df["epoch"] = history_df.index + 1

# Plotte die Loss (Mean Squared Error)
fig_loss = px.line(
    history_df,
    x="epoch",
    y=["loss", "val_loss"],
    title="Trainings- & Validierungs-Loss (MSE)",
    labels={"value": "Loss (MSE)", "variable": "Datensatz"},
)
fig_loss.show()

# Plotte die Metrik (Mean Absolute Error)
fig_mae = px.line(
    history_df,
    x="epoch",
    y=["mean_absolute_error", "val_mean_absolute_error"],
    title="Trainings- & Validierungs-MAE",
    labels={"value": "Mean Absolute Error", "variable": "Datensatz"},
)
fig_mae.show()

In [16]:
# EVALUIERUNG


# 1. Lade das beste Modell, das während des Trainings gespeichert wurde
print("Lade das beste gespeicherte Modell 'best_model.keras'...")
best_model = load_model("best_model.keras")
print("Modell geladen.")

# 2. Definiere die Schritte für die Evaluierung (z.B. in 10%-Schritten)
evaluation_points = np.linspace(0.1, 1.0, 10)  # 10%, 20%, ..., 100%
timesteps = X_test_scaled.shape[1]  # 140
mae_per_step = []

print(
    "\nStarte die Evaluierung der Vorhersagegenauigkeit in Abhängigkeit vom Batch-Fortschritt..."
)

for progress in evaluation_points:
    # Berechne, wie viele Zeitschritte dem aktuellen Fortschritt entsprechen
    current_timesteps = int(timesteps * progress)

    # Erstelle eine temporäre, verkürzte Version des Testsets
    X_test_partial = X_test_scaled[:, :current_timesteps, :]

    # Padde die Sequenzen von vorne mit Nullen, damit sie die volle Länge haben
    # Das ist wichtig, da das LSTM eine feste Input-Länge erwartet.
    # 'pre'-padding ist üblich für Zeitreihen.
    X_test_padded = pad_sequences(
        X_test_partial,
        maxlen=timesteps,
        dtype="float32",
        padding="pre",
        truncating="pre",
    )

    # Mache Vorhersagen mit dem Modell auf den (skalierten) gepaddeten Daten
    y_pred_scaled = best_model.predict(X_test_padded)

    # Rück-Transformation der Vorhersagen und der wahren Werte in die Originalskala
    y_pred = scaler_y.inverse_transform(y_pred_scaled)
    y_true = scaler_y.inverse_transform(
        y_test_scaled
    )  # y_test_scaled sind die Targets für X_test_scaled

    # Berechne den Mean Absolute Error für diesen Fortschrittsschritt
    mae = mean_absolute_percentage_error(y_true, y_pred)
    mae_per_step.append(mae)

    print(
        f"Fortschritt: {progress*100:.0f}% ({current_timesteps} Zeitschritte) -> MAE: {mae:.4f}"
    )

print("\nEvaluierung abgeschlossen.")


# 3. Visualisierung des Ergebnisses
eval_df = pd.DataFrame(
    {
        "Batch-Fortschritt (%)": evaluation_points * 100,
        "Mean Absolute Percent Error (MAPE)": mae_per_step,
    }
)

fig_eval = px.line(
    eval_df,
    x="Batch-Fortschritt (%)",
    y="Mean Absolute Percent Error (MAPE)",
    title="Soft-Sensor-Genauigkeit während des Batches",
    markers=True,
)

fig_eval.update_layout(
    yaxis_title="Vorhersagefehler (MAPE) auf dem Testset",
    xaxis_title="Verfügbare Daten vom Batch (%)",
)
fig_eval.show()

Lade das beste gespeicherte Modell 'best_model.keras'...
Modell geladen.

Starte die Evaluierung der Vorhersagegenauigkeit in Abhängigkeit vom Batch-Fortschritt...
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step
Fortschritt: 10% (14 Zeitschritte) -> MAE: 0.2032
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
Fortschritt: 20% (28 Zeitschritte) -> MAE: 0.2223
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
Fortschritt: 30% (42 Zeitschritte) -> MAE: 0.2394
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
Fortschritt: 40% (56 Zeitschritte) -> MAE: 0.2391
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
Fortschritt: 50% (70 Zeitschritte) -> MAE: 0.1708
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
Fortschritt: 60% (84 Zeitschritte) -> MAE: 0.1837
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
Fortschritt: 70% (98 Zeitschritte) -> MAE: 0.1835
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
Fortschritt: 80% (112 Zeitschritte) -> MAE: 0.1355
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
Fortschritt: 90% (126 Zeitschritte) -> MAE: 0.0648
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
Fort

1. Wir wählen einen Beispiel-Batch aus unserem Testset (dies simuliert einen "neuen, unbekannten" Batch).
2. Wir "spulen die Zeit vor": Wir nehmen zuerst nur die ersten 10% der Daten dieses Batches, dann die ersten 20%, und so weiter.
3. Wir füttern das Modell mit diesen unvollständigen Daten und lassen es eine Vorhersage für den Endwert treffen.
4. Wir vergleichen die Vorhersage mit dem wahren, aber eigentlich "unbekannten" Endwert dieses Batches.

In [24]:
import random

In [65]:
# VORHERSAGE WÄHREND EINES BATCHES


# 1. SETUP: Lade Modell und Scaler und wähle einen Beispiel-Batch aus

# Lade das trainierte Modell und die Scaler (stellen Sie sicher, dass die vorherigen Zellen gelaufen sind)
model = load_model("best_model.keras")
# scaler_X und scaler_y sind bereits aus der Pipeline-Zelle im Speicher

# Wähle einen zufälligen Batch aus dem Testset für unsere Demonstration
# Wir nehmen den Batch mit dem Index 5 aus dem Testset (indices_test[5])
# Sie können hier jeden Index von 0 bis len(X_test_scaled)-1 wählen
sample_index = 5
# sample_index = random.randint(0, len(X_test_scaled) - 1)
sample_batch_scaled = X_test_scaled[sample_index]
sample_target_scaled = y_test_scaled[sample_index]

# Hol den wahren Endwert und skaliere ihn zurück, um ihn später vergleichen zu können
true_final_value = scaler_y.inverse_transform(sample_target_scaled.reshape(1, -1))[0, 0]

print(f"Demonstration für einen einzelnen Batch aus dem Testset.")
print(
    f"Der wahre Endwert (Qualitätsvariable) für diesen Batch ist: {true_final_value:.2f}"
)
print("-" * 50)


# 2. DER VORHERSAGE-PROZESS

# Wir machen Vorhersagen zu verschiedenen Zeitpunkten
evaluation_points = np.linspace(0.1, 1.0, 10)  # 10%, 20%, ..., 100%
timesteps = sample_batch_scaled.shape[0]  # 140
predictions_over_time = []

for progress in evaluation_points:
    # Nimm nur die bisher verfügbaren Daten des Batches
    current_timesteps = int(timesteps * progress)
    partial_sequence = sample_batch_scaled[:current_timesteps, :]

    # Das Modell erwartet immer einen Input der Länge 140.
    # Wir müssen die unvollständige Sequenz mit Nullen auffüllen (Padding).
    # Wir fügen die Nullen am Anfang hinzu ('pre'), da die neuesten Daten am wichtigsten sind.
    padded_sequence = pad_sequences(
        [partial_sequence],  # Muss in einer Liste sein
        maxlen=timesteps,
        dtype="float32",
        padding="pre",
    )

    # Mache die Vorhersage. Das `[0, 0]` am Ende extrahiert den einzelnen Zahlenwert.
    prediction_scaled = model.predict(padded_sequence)[0, 0]

    # Skaliere die Vorhersage zurück in die ursprüngliche, interpretierbare Einheit
    prediction_real_value = scaler_y.inverse_transform([[prediction_scaled]])[0, 0]

    predictions_over_time.append(prediction_real_value)

    print(
        f"Nach {progress*100:.0f}% des Batches: Vorhersage für Endwert = {prediction_real_value:.2f}"
    )


# 3. VISUALISIERUNG DES ERGEBNISSES

# Erstelle ein DataFrame für den Plot
prediction_df = pd.DataFrame(
    {
        "Batch-Fortschritt (%)": evaluation_points * 100,
        "Vorhergesagter Endwert": predictions_over_time,
    }
)

# Erstelle die Figur
fig = go.Figure()

# Füge die Linie für die Vorhersagen hinzu
fig.add_trace(
    go.Scatter(
        x=prediction_df["Batch-Fortschritt (%)"],
        y=prediction_df["Vorhergesagter Endwert"],
        mode="lines+markers",
        name="Soft-Sensor-Vorhersage",
    )
)

# Füge eine horizontale Linie für den wahren Endwert hinzu
fig.add_hline(
    y=true_final_value,
    line_dash="dash",
    line_color="red",
    annotation_text=f"Wahrer Endwert: {true_final_value}",
    annotation_position="bottom right",
)

fig.update_layout(
    title="Live-Vorhersage des Soft Sensors für einen einzelnen Batch",
    xaxis_title="Verfügbare Daten vom Batch (%)",
    yaxis_title="Vorhergesagtes Gesamtvolumen V_total_out",
)

fig.show()

Demonstration für einen einzelnen Batch aus dem Testset.
Der wahre Endwert (Qualitätsvariable) für diesen Batch ist: 853.83
--------------------------------------------------
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step
Nach 10% des Batches: Vorhersage für Endwert = 939.87
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
Nach 20% des Batches: Vorhersage für Endwert = 952.70
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
Nach 30% des Batches: Vorhersage für Endwert = 951.49
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
Nach 40% des Batches: Vorhersage für Endwert = 915.95
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
Nach 50% des Batches: Vorhersage für Endwert = 758.40
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
Nach 60% des Batches: Vorhersage für Endwert = 618.87
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
Nach 70% des Batches: Vorhersage für Endwert = 615.73
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
Nach 80% des Batches: Vorhersage für Endwert = 718.75
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
Nach 90% des Batches: Vorhersage für Endwert = 801.

In [64]:
res[res["V_total_out"] == 853.8283193238688]

,t,Sequenz,u1(t),u2(t),x1(t),x2(t),x3(t),v(t),V_total_out
23100,0,166,0.000000,0.000000,3.324964,0.314989,1.895498,6.273484,853.828319
23101,1,166,0.529042,1.043326,1.830373,1.808865,1.938154,6.125705,853.828319
23102,2,166,1.057493,2.010149,1.824262,2.110489,2.034813,6.032023,853.828319
23103,3,166,1.584777,2.903901,2.092122,1.989551,2.000470,5.927173,853.828319
23104,4,166,2.110335,3.727952,2.102525,1.868884,2.175751,6.211442,853.828319
...,...,...,...,...,...,...,...,...,...
23235,135,166,7.906599,1.746841,4.175363,4.122193,0.560810,3.057802,853.828319
23236,136,166,7.303199,1.342170,4.351235,4.505727,0.704037,2.661799,853.828319
23237,137,166,6.663780,0.971866,4.583188,4.201297,0.403847,2.117710,853.828319
23238,138,166,5.986375,0.640411,4.233776,4.387759,0.176325,1.237216,853.828319


unterschied Differenz Füllstände